# Before fine-tuning

### Data format
- phải ở dạng jsonl
- là một danh sách chứa các messages, trong đó chứa role và content
- (là một danh sách các đối tượng messages, trong đó một message chứa một danh sách các phiên trò chuyện -> xem ví dụ bên dưới)
- role chỉ chấp nhận ba trường hợp: system, user, chatbot
- content chứa nội dung của message cụ thể

### Data Requirements
để vượt qua các bài kiểm tra xác thực mà cohere thực hiện trên dữ liệu đã tải lên, phải đảm bảo rằng:
- chỉ có 3 role chuẩn được chấp nhận kể trên
- user phải có ít nhất một trường hợp của chatbot và người dùng
- phần mở đầu phải được tải lên làm tin nhắn đầu tiên trong cuộc trò chuyện, với vai trò role: system (1)
- tất cả các tin nhắn khác có vai trò role: system, sẽ được coi là người nói trong cuộc trò chuyện (1)
(1) hai mục này ý nói đến, trong một message, role system đầu tiên phải được tải lên, các role system khác có sau đó (có thể không cần thiết) nếu có thì sẽ được xem như là một speaker
- mỗi phiên trò chuyện (message), ngữ cảnh không dài quá 16348 tokens để tránh bị loại khỏi tập dữ liệu
- dữ liệu mã hóa dạng utf-8
- `Dữ liệu của bạn phải nằm trong tệp .jsonl, trong đó mỗi đối tượng json là một cuộc hội thoại có cấu trúc sau:`
{
  "messages": [

        {
        "role": "system",
        "content": "You are a chatbot trained to answer to my every question."
        },

        {
        "role": "user",
        "content": "Hello"
        },

        {
        "role": "chatbot",
        "content": "Greetings! How can I help you?"
        }, ...

  ]
}



### Evaluation Datasets
- dữ liệu evaludate được dùng để đánh giá hiệu suất mô hình đào tạo
- có thể cung cấp dữ liệu đánh giá riêng biệt hoặc cho phép tự chia theo tỉ lệ

### Create a Dataset with the Python SDK
(nếu tải dataset lên bằng UI thì bỏ qua phần này)

In [ ]:
import cohere

# instantiate the Cohere client
co = cohere.ClientV2("YOUR_API_KEY")

chat_dataset = co.datasets.create(
    name="chat-dataset",
    data=open("path/to/train.jsonl", "rb"),
    type="chat-finetune-input",
)
print(co.wait(chat_dataset))

chat_dataset_with_eval = co.datasets.create(
    name="chat-dataset-with-eval",
    data=open("path/to/train.jsonl", "rb"),
    eval_data=open("path/to/eval.jsonl", "rb"),
    type="chat-finetune-input",
)
print(co.wait(chat_dataset_with_eval))


### Parameters

`hyperparameters (cohere.finetuning.Hyperparameters)` được dùng để điều chỉnh siêu tham số cho quá trình huấn luyện
- train_epochs (int)
- learning_rate (float)
- train_batch_size (int)
- early_stopping_threshold (float)
- early_stopping_patience (int)

### Code example

In [ ]:
import cohere
from cohere.finetuning import (
    Hyperparameters,
    Settings,
    FinetunedModel,
    BaseModel,
    WandbConfig,
)

co = cohere.ClientV2("Your API key")

chat_dataset = co.datasets.create(
    name="chat-dataset",
    data=open("path/to/train.jsonl", "rb"),
    type="chat-finetune-input",
)
# optional (define custom hyperparameters)
hp = Hyperparameters(
    early_stopping_patience=10,
    early_stopping_threshold=0.001,
    train_batch_size=16,
    train_epochs=1,
    learning_rate=0.01,
)

# optional (define wandb configuration)
wnb_config = WandbConfig(
    project="test-project",
    api_key="<<wandbApiKey>>",
    entity="test-entity",
)

create_response = co.finetuning.create_finetuned_model(
    request=FinetunedModel(
        name="customer-service-chat-model",
        settings=Settings(
            base_model=BaseModel(
                base_type="BASE_TYPE_CHAT",
            ),
            dataset_id=chat_dataset.id,
            hyperparameters=hp,
            wandb=wnb_config,
        ),
    ),
)


# Fine-tuning LLM model